# Deliverable 1: Preprocessing the Data for a Neural Network

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("./Resources/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(['EIN','NAME','ORGANIZATION'],1)
application_df.head()

C:\Users\rfamo\anaconda3\envs\mlenv\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  


,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,1,100000-499999,N,142590,1


In [3]:
# Determine the number of unique values in each column.
application_df.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [4]:
# Look at APPLICATION_TYPE value counts for binning
application_count = application_df.APPLICATION_TYPE.value_counts()
application_count

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [5]:
# Determine which values to replace if counts are less than ...?
replace_application = list(application_count[application_count < 500].index)

# Replace in dataframe
for app in replace_application:
    application_df.APPLICATION_TYPE = application_df.APPLICATION_TYPE.replace(app,"Other")
    
# Check to make sure binning was successful
application_df.APPLICATION_TYPE.value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [6]:
# Look at CLASSIFICATION value counts for binning
classification_count = application_df.CLASSIFICATION.value_counts()
classification_count

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [7]:
# Determine which values to replace if counts are less than ..?
replace_class = list(classification_count[classification_count < 1800].index)

# Replace in dataframe
for cls in replace_class:
    application_df.CLASSIFICATION = application_df.CLASSIFICATION.replace(cls,"Other")
    
# Check to make sure binning was successful
application_df.CLASSIFICATION.value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [8]:
# Look at INCOME_AMT value counts for binning
income_count = application_df.INCOME_AMT.value_counts()
income_count

0                24388
25000-99999       3747
100000-499999     3374
1M-5M              955
1-9999             728
10000-24999        543
10M-50M            240
5M-10M             185
50M+               139
Name: INCOME_AMT, dtype: int64

In [9]:
# Determine which values to replace if counts are less than ..?
replace_income = list(income_count[income_count < 500].index)

# Replace in dataframe
for cls in replace_income:
    application_df.INCOME_AMT = application_df.INCOME_AMT.replace(cls,"Other")
    
# Check to make sure binning was successful
application_df.INCOME_AMT.value_counts()

0                24388
25000-99999       3747
100000-499999     3374
1M-5M              955
1-9999             728
Other              564
10000-24999        543
Name: INCOME_AMT, dtype: int64

In [10]:
# Generate our categorical variable lists
application_cat = application_df.dtypes[application_df.dtypes == 'object'].index.tolist()
application_df[application_cat].nunique()

APPLICATION_TYPE          9
AFFILIATION               6
CLASSIFICATION            6
USE_CASE                  5
INCOME_AMT                7
SPECIAL_CONSIDERATIONS    2
dtype: int64

In [11]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(application_df[application_cat]))

# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names(application_cat)
encode_df.head()

C:\Users\rfamo\anaconda3\envs\mlenv\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,APPLICATION_TYPE_T8,AFFILIATION_CompanySponsored,...,USE_CASE_ProductDev,INCOME_AMT_0,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_Other,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0


In [12]:
# Merge one-hot encoded features and drop the originals
application_df = application_df.merge(encode_df, left_index=True, right_index=True)
application_df = application_df.drop(application_cat,1)
application_df.head()

C:\Users\rfamo\anaconda3\envs\mlenv\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  This is separate from the ipykernel package so we can avoid doing imports until


,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,...,USE_CASE_ProductDev,INCOME_AMT_0,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_Other,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,1,108590,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,1,5000,0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,1,6692,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
4,1,142590,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0


In [13]:
# Split our preprocessed data into our features and target arrays
y = application_df['IS_SUCCESSFUL'].values.reshape(-1,1)
X = application_df.drop(['IS_SUCCESSFUL','SPECIAL_CONSIDERATIONS_Y'],1).values

# Split the preprocessed data into a training and testing dataset
X_train,X_test,y_train,y_test = train_test_split(X,y,random_state=42, stratify=y)

C:\Users\rfamo\anaconda3\envs\mlenv\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  This is separate from the ipykernel package so we can avoid doing imports until


# Deliverable 2: Compile, Train and Evaluate the Model

In [15]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.

number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 = 100
hidden_nodes_layer2 = 50

nn = Sequential()

# First hidden layer
nn.add(Dense(units=hidden_nodes_layer1,input_dim=number_input_features,activation='relu'))

# Second hidden layer
nn.add(Dense(units=hidden_nodes_layer2,activation='relu'))

# Output layer
nn.add(Dense(units=1,activation='sigmoid'))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 100)               3700      
                                                                 
 dense_1 (Dense)             (None, 50)                5050      
                                                                 
 dense_2 (Dense)             (None, 1)                 51        
                                                                 
Total params: 8,801
Trainable params: 8,801
Non-trainable params: 0
_________________________________________________________________


In [16]:
# Compile the model
nn.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [17]:
# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=100)

Epoch 1/100
804/804 [==============================] - 3s 2ms/step - loss: 0.5765 - accuracy: 0.7187
Epoch 2/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5647 - accuracy: 0.7254
Epoch 3/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5615 - accuracy: 0.7275
Epoch 4/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5600 - accuracy: 0.7272
Epoch 5/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5592 - accuracy: 0.7301
Epoch 6/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5580 - accuracy: 0.7282
Epoch 7/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5584 - accuracy: 0.7292
Epoch 8/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5576 - accuracy: 0.7290
Epoch 9/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5566 - accuracy: 0.7289
Epoch 10/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5567 - accura

In [18]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5759 - accuracy: 0.7230 - 495ms/epoch - 2ms/step
Loss: 0.5759226083755493, Accuracy: 0.7230320572853088


In [19]:
# Import checkpoint dependencies
import os
from tensorflow.keras.callbacks import ModelCheckpoint

# Define the checkpoint path and filenames
os.makedirs('checkpoints/Optimization2/',exist_ok=True)
checkpoint_path = 'checkpoints/Optimization2/weights.{epoch:02d}.hdf5'

# Create a callback that saves the model's weights every 5 epochs
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq=100)

# Compile the model
nn.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=100,callbacks=[cp_callback])

Epoch 1/100
 66/804 [=>............................] - ETA: 1s - loss: 0.5500 - accuracy: 0.7306
Epoch 1: saving model to checkpoints/Optimization2\weights.01.hdf5
190/804 [======>.......................] - ETA: 1s - loss: 0.5484 - accuracy: 0.7332
Epoch 1: saving model to checkpoints/Optimization2\weights.01.hdf5
274/804 [=========>....................] - ETA: 1s - loss: 0.5446 - accuracy: 0.7373
Epoch 1: saving model to checkpoints/Optimization2\weights.01.hdf5
378/804 [=============>................] - ETA: 0s - loss: 0.5472 - accuracy: 0.7360
Epoch 1: saving model to checkpoints/Optimization2\weights.01.hdf5
483/804 [=================>............] - ETA: 0s - loss: 0.5500 - accuracy: 0.7328
Epoch 1: saving model to checkpoints/Optimization2\weights.01.hdf5
582/804 [====================>.........] - ETA: 0s - loss: 0.5512 - accuracy: 0.7322
Epoch 1: saving model to checkpoints/Optimization2\weights.01.hdf5
690/804 [========================>.....] - ETA: 0s - loss: 0.5499 - accuracy

272/804 [=========>....................] - ETA: 1s - loss: 0.5488 - accuracy: 0.7361
Epoch 7: saving model to checkpoints/Optimization2\weights.07.hdf5
357/804 [============>.................] - ETA: 1s - loss: 0.5475 - accuracy: 0.7346
Epoch 7: saving model to checkpoints/Optimization2\weights.07.hdf5
451/804 [===============>..............] - ETA: 0s - loss: 0.5467 - accuracy: 0.7345
Epoch 7: saving model to checkpoints/Optimization2\weights.07.hdf5
571/804 [====================>.........] - ETA: 0s - loss: 0.5479 - accuracy: 0.7334
Epoch 7: saving model to checkpoints/Optimization2\weights.07.hdf5
672/804 [========================>.....] - ETA: 0s - loss: 0.5479 - accuracy: 0.7338
Epoch 7: saving model to checkpoints/Optimization2\weights.07.hdf5
765/804 [===========================>..] - ETA: 0s - loss: 0.5485 - accuracy: 0.7332
Epoch 7: saving model to checkpoints/Optimization2\weights.07.hdf5
804/804 [==============================] - 2s 2ms/step - loss: 0.5485 - accuracy: 0.7334

439/804 [===============>..............] - ETA: 0s - loss: 0.5472 - accuracy: 0.7355
Epoch 13: saving model to checkpoints/Optimization2\weights.13.hdf5
537/804 [===================>..........] - ETA: 0s - loss: 0.5464 - accuracy: 0.7357
Epoch 13: saving model to checkpoints/Optimization2\weights.13.hdf5
641/804 [======================>.......] - ETA: 0s - loss: 0.5469 - accuracy: 0.7357
Epoch 13: saving model to checkpoints/Optimization2\weights.13.hdf5
748/804 [==========================>...] - ETA: 0s - loss: 0.5493 - accuracy: 0.7330
Epoch 13: saving model to checkpoints/Optimization2\weights.13.hdf5
804/804 [==============================] - 2s 2ms/step - loss: 0.5487 - accuracy: 0.7334
Epoch 14/100
 29/804 [>.............................] - ETA: 1s - loss: 0.5588 - accuracy: 0.7371
Epoch 14: saving model to checkpoints/Optimization2\weights.14.hdf5
136/804 [====>.........................] - ETA: 1s - loss: 0.5523 - accuracy: 0.7289
Epoch 14: saving model to checkpoints/Optimizati

592/804 [=====================>........] - ETA: 0s - loss: 0.5508 - accuracy: 0.7313
Epoch 25: saving model to checkpoints/Optimization2\weights.25.hdf5
702/804 [=========================>....] - ETA: 0s - loss: 0.5485 - accuracy: 0.7337
Epoch 25: saving model to checkpoints/Optimization2\weights.25.hdf5
787/804 [============================>.] - ETA: 0s - loss: 0.5479 - accuracy: 0.7341
Epoch 25: saving model to checkpoints/Optimization2\weights.25.hdf5
804/804 [==============================] - 2s 2ms/step - loss: 0.5482 - accuracy: 0.7339
Epoch 26/100
 78/804 [=>............................] - ETA: 1s - loss: 0.5576 - accuracy: 0.7280
Epoch 26: saving model to checkpoints/Optimization2\weights.26.hdf5
179/804 [=====>........................] - ETA: 1s - loss: 0.5555 - accuracy: 0.7308
Epoch 26: saving model to checkpoints/Optimization2\weights.26.hdf5
286/804 [=========>....................] - ETA: 1s - loss: 0.5581 - accuracy: 0.7287
Epoch 26: saving model to checkpoints/Optimizati

755/804 [===========================>..] - ETA: 0s - loss: 0.5468 - accuracy: 0.7352
Epoch 37: saving model to checkpoints/Optimization2\weights.37.hdf5
804/804 [==============================] - 1s 2ms/step - loss: 0.5478 - accuracy: 0.7341
Epoch 38/100
 35/804 [>.............................] - ETA: 1s - loss: 0.5515 - accuracy: 0.7277
Epoch 38: saving model to checkpoints/Optimization2\weights.38.hdf5
144/804 [====>.........................] - ETA: 1s - loss: 0.5407 - accuracy: 0.7428
Epoch 38: saving model to checkpoints/Optimization2\weights.38.hdf5
247/804 [========>.....................] - ETA: 0s - loss: 0.5456 - accuracy: 0.7377
Epoch 38: saving model to checkpoints/Optimization2\weights.38.hdf5
340/804 [===========>..................] - ETA: 0s - loss: 0.5455 - accuracy: 0.7359
Epoch 38: saving model to checkpoints/Optimization2\weights.38.hdf5
428/804 [==============>...............] - ETA: 0s - loss: 0.5460 - accuracy: 0.7361
Epoch 38: saving model to checkpoints/Optimizati

 77/804 [=>............................] - ETA: 1s - loss: 0.5460 - accuracy: 0.7399
Epoch 50: saving model to checkpoints/Optimization2\weights.50.hdf5
195/804 [======>.......................] - ETA: 1s - loss: 0.5447 - accuracy: 0.7383
Epoch 50: saving model to checkpoints/Optimization2\weights.50.hdf5
297/804 [==========>...................] - ETA: 0s - loss: 0.5467 - accuracy: 0.7367
Epoch 50: saving model to checkpoints/Optimization2\weights.50.hdf5
373/804 [============>.................] - ETA: 0s - loss: 0.5485 - accuracy: 0.7348
Epoch 50: saving model to checkpoints/Optimization2\weights.50.hdf5
488/804 [=================>............] - ETA: 0s - loss: 0.5502 - accuracy: 0.7328
Epoch 50: saving model to checkpoints/Optimization2\weights.50.hdf5
579/804 [====================>.........] - ETA: 0s - loss: 0.5505 - accuracy: 0.7325
Epoch 50: saving model to checkpoints/Optimization2\weights.50.hdf5
691/804 [========================>.....] - ETA: 0s - loss: 0.5474 - accuracy: 0.73

232/804 [=======>......................] - ETA: 1s - loss: 0.5443 - accuracy: 0.7349
Epoch 62: saving model to checkpoints/Optimization2\weights.62.hdf5
340/804 [===========>..................] - ETA: 0s - loss: 0.5405 - accuracy: 0.7378
Epoch 62: saving model to checkpoints/Optimization2\weights.62.hdf5
444/804 [===============>..............] - ETA: 0s - loss: 0.5420 - accuracy: 0.7378
Epoch 62: saving model to checkpoints/Optimization2\weights.62.hdf5
550/804 [===================>..........] - ETA: 0s - loss: 0.5447 - accuracy: 0.7356
Epoch 62: saving model to checkpoints/Optimization2\weights.62.hdf5
632/804 [======================>.......] - ETA: 0s - loss: 0.5461 - accuracy: 0.7341
Epoch 62: saving model to checkpoints/Optimization2\weights.62.hdf5
739/804 [==========================>...] - ETA: 0s - loss: 0.5473 - accuracy: 0.7338
Epoch 62: saving model to checkpoints/Optimization2\weights.62.hdf5
804/804 [==============================] - 2s 2ms/step - loss: 0.5477 - accuracy: 

394/804 [=============>................] - ETA: 0s - loss: 0.5463 - accuracy: 0.7346
Epoch 74: saving model to checkpoints/Optimization2\weights.74.hdf5
501/804 [=================>............] - ETA: 0s - loss: 0.5454 - accuracy: 0.7365
Epoch 74: saving model to checkpoints/Optimization2\weights.74.hdf5
607/804 [=====================>........] - ETA: 0s - loss: 0.5461 - accuracy: 0.7362
Epoch 74: saving model to checkpoints/Optimization2\weights.74.hdf5
688/804 [========================>.....] - ETA: 0s - loss: 0.5471 - accuracy: 0.7350
Epoch 74: saving model to checkpoints/Optimization2\weights.74.hdf5
804/804 [==============================] - 2s 2ms/step - loss: 0.5477 - accuracy: 0.7345
Epoch 75/100
  1/804 [..............................] - ETA: 2s - loss: 0.3927 - accuracy: 0.9062
Epoch 75: saving model to checkpoints/Optimization2\weights.75.hdf5
 78/804 [=>............................] - ETA: 1s - loss: 0.5404 - accuracy: 0.7396
Epoch 75: saving model to checkpoints/Optimizati

550/804 [===================>..........] - ETA: 0s - loss: 0.5437 - accuracy: 0.7353
Epoch 86: saving model to checkpoints/Optimization2\weights.86.hdf5
651/804 [=======================>......] - ETA: 0s - loss: 0.5449 - accuracy: 0.7357
Epoch 86: saving model to checkpoints/Optimization2\weights.86.hdf5
757/804 [===========================>..] - ETA: 0s - loss: 0.5453 - accuracy: 0.7355
Epoch 86: saving model to checkpoints/Optimization2\weights.86.hdf5
804/804 [==============================] - 1s 2ms/step - loss: 0.5467 - accuracy: 0.7345
Epoch 87/100
 30/804 [>.............................] - ETA: 1s - loss: 0.5405 - accuracy: 0.7427
Epoch 87: saving model to checkpoints/Optimization2\weights.87.hdf5
141/804 [====>.........................] - ETA: 1s - loss: 0.5473 - accuracy: 0.7296
Epoch 87: saving model to checkpoints/Optimization2\weights.87.hdf5
255/804 [========>.....................] - ETA: 1s - loss: 0.5524 - accuracy: 0.7271
Epoch 87: saving model to checkpoints/Optimizati

702/804 [=========================>....] - ETA: 0s - loss: 0.5468 - accuracy: 0.7339
Epoch 98: saving model to checkpoints/Optimization2\weights.98.hdf5
804/804 [==============================] - 1s 2ms/step - loss: 0.5472 - accuracy: 0.7343
Epoch 99/100
  1/804 [..............................] - ETA: 2s - loss: 0.5850 - accuracy: 0.7188
Epoch 99: saving model to checkpoints/Optimization2\weights.99.hdf5
 81/804 [==>...........................] - ETA: 1s - loss: 0.5524 - accuracy: 0.7288
Epoch 99: saving model to checkpoints/Optimization2\weights.99.hdf5
194/804 [======>.......................] - ETA: 1s - loss: 0.5405 - accuracy: 0.7387
Epoch 99: saving model to checkpoints/Optimization2\weights.99.hdf5
299/804 [==========>...................] - ETA: 0s - loss: 0.5429 - accuracy: 0.7362
Epoch 99: saving model to checkpoints/Optimization2\weights.99.hdf5
378/804 [=============>................] - ETA: 0s - loss: 0.5444 - accuracy: 0.7359
Epoch 99: saving model to checkpoints/Optimizati

In [20]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5944 - accuracy: 0.7231 - 402ms/epoch - 1ms/step
Loss: 0.594367265701294, Accuracy: 0.7231487035751343


In [21]:
# Export our model to HDF5 file
nn.save('AlphabetSoupCharity_Optimization2.h5')